In [1]:
#hide
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()
from fastbook import *

import pandas as pd
import numpy as np
from torch.nn.functional import nll_loss,log_softmax

from sklearn import preprocessing
from functools import partial
import copy

In [13]:
file = "training/training-observations-v0-2.csv"
obs = pd.read_csv(file)
obs

,gbifid,_family,genus,species,_year,_month,_day,eventdate,decimallatitude,decimallongitude,kg,elu,elu_class1,elu_class2,elu_class3
0,1438086806,Polyporaceae,Trametes,Trametes ochracea,2014,12.0,7.0,2014-12-07 00:00:00,58.448580,13.701640,26,372,Cold Wet Hills,Carbonate Sedimentary Rock,Mostly Deciduous Forest
1,1669971044,Stereocaulaceae,Hertelidea,Hertelidea botryosa,2012,5.0,21.0,2012-05-21 00:00:00,59.551380,12.236730,27,301,Cold Wet Hills,Metamorphic Rock,Mostly Needleleaf/Evergreen Forest
2,2271059142,Ramalinaceae,Lecania,Lecania naegelii,2018,9.0,12.0,2018-09-12 00:00:00,59.262680,15.548110,26,451,Cold Wet Plains,Acidic Plutonics,Mostly Cropland
3,2865967870,Russulaceae,Russula,Russula aeruginea,2020,9.0,24.0,2020-09-24 00:00:00,59.487460,14.942440,26,230,Cold Wet Hills,Carbonate Sedimentary Rock,Mostly Needleleaf/Evergreen Forest
4,2983765339,Polyporaceae,Leptoporus,Leptoporus mollis,2020,11.0,8.0,2020-11-08 00:00:00,59.789860,17.734220,26,162,Cold Wet Plains,Acidic Plutonics,Mostly Needleleaf/Evergreen Forest
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11667436,2627015371,Lobariaceae,Ricasolia,Ricasolia virens,2020,3.0,22.0,2020-03-22 00:00:00,62.974435,8.068344,26,0,NaN,NaN,NaN
11667437,855161200,Strophariaceae,Hypholoma,Hypholoma fasciculare,2007,10.0,5.0,2007-10-05 00:00:00,58.323460,11.850640,26,0,NaN,NaN,NaN
11667438,1302816506,Parmeliaceae,Parmelia,Parmelia omphalodes,1925,7.0,31.0,1925-07-31 00:00:00,70.153100,19.667100,27,0,NaN,NaN,NaN
11667439,3080077525,Polyporaceae,Cerrena,Cerrena unicolor,2021,3.0,6.0,2021-03-06 00:00:00,63.114399,7.806971,15,0,NaN,NaN,NaN


In [15]:
data = obs[['species', '_month', '_day', 'eventdate', 'decimallatitude', 'decimallongitude', 'kg', 'elu', 'elu_class1', 'elu_class2', 'elu_class3']]
data

,species,_month,_day,eventdate,decimallatitude,decimallongitude,kg,elu,elu_class1,elu_class2,elu_class3
0,Trametes ochracea,12.0,7.0,2014-12-07 00:00:00,58.448580,13.701640,26,372,Cold Wet Hills,Carbonate Sedimentary Rock,Mostly Deciduous Forest
1,Hertelidea botryosa,5.0,21.0,2012-05-21 00:00:00,59.551380,12.236730,27,301,Cold Wet Hills,Metamorphic Rock,Mostly Needleleaf/Evergreen Forest
2,Lecania naegelii,9.0,12.0,2018-09-12 00:00:00,59.262680,15.548110,26,451,Cold Wet Plains,Acidic Plutonics,Mostly Cropland
3,Russula aeruginea,9.0,24.0,2020-09-24 00:00:00,59.487460,14.942440,26,230,Cold Wet Hills,Carbonate Sedimentary Rock,Mostly Needleleaf/Evergreen Forest
4,Leptoporus mollis,11.0,8.0,2020-11-08 00:00:00,59.789860,17.734220,26,162,Cold Wet Plains,Acidic Plutonics,Mostly Needleleaf/Evergreen Forest
...,...,...,...,...,...,...,...,...,...,...,...
11667436,Ricasolia virens,3.0,22.0,2020-03-22 00:00:00,62.974435,8.068344,26,0,NaN,NaN,NaN
11667437,Hypholoma fasciculare,10.0,5.0,2007-10-05 00:00:00,58.323460,11.850640,26,0,NaN,NaN,NaN
11667438,Parmelia omphalodes,7.0,31.0,1925-07-31 00:00:00,70.153100,19.667100,27,0,NaN,NaN,NaN
11667439,Cerrena unicolor,3.0,6.0,2021-03-06 00:00:00,63.114399,7.806971,15,0,NaN,NaN,NaN


In [10]:
family_map = {species:family for (species,_,family) in obs.groupby(by=["species","genus","_family"]).indices.keys()}
family_list = list(family_map.values())
fle = preprocessing.LabelEncoder()
family_targets = torch.tensor(fle.fit_transform(family_list), device='cuda:0')
family_dims = family_targets.max().int().item() + 1

genus_map = {species:genus for (species,genus,_) in obs.groupby(by=["species","genus","_family"]).indices.keys()}
genus_list = list(genus_map.values())
gle = preprocessing.LabelEncoder()
genus_targets = torch.tensor(gle.fit_transform(genus_list), device='cuda:0')
genus_dims = genus_targets.max().int().item() + 1